<a href="https://colab.research.google.com/github/oussemajelassi/embedded_ReinforcementLearning/blob/main/RL_RobotOrientation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This project aims to perform robot orientation using RL

In [ ]:
# Install required modules
!python -m pip install gymnasium==0.28.1
!python -m pip install stable-baselines3[extra]==2.1.0
!python -m pip install pyserial

In [ ]:
# Import Modules
import gymnasium as gym
import stable_baselines3 as sb3
from gymnasium import spaces


import numpy as np
import pygame
import time
import json

import serial
import serial.tools.list_ports
import time

c:\users\ousjl\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Vairbales and Constants :

STP_ACTIONS_MAP = {
    0: -100,
    1: 0,
    2: 100,
                  }

STM_ResponseTimeout = 150

# Test Block for Json parsing :


In [ ]:
msg = "{ \"message\": \"Hello, world!\", \"sender\": \"John Doe\", \"timestamp\": \"2024-01-22T12:34:56\" }"
obs = json.loads(msg)
print (obs['message'])

Hello, world!


## Test for serial communcation  



In [ ]:
serial_list = []
for port, desc, hwid in serial.tools.list_ports.comports():
  serial_attrs = (port, desc, hwid)
  serial_list.append(serial_attrs)
  print(serial_list)

## Low Level communication Class

In [ ]:
class LowLevelCommunication :
  def __init__(self , Port , Baudrate) -> bool:
    self.STM32 = serial.Serial(dsrdtr=False)
    self.STM32.port = Port
    self.STM32.baudrate = Baudrate
    self.STM32.rts = False
    self.STM32.dtr = False
    self.STM32.close()
    time.sleep(1)
    self.STM32.open()
    print("Open")

  def sendCommand (self , Command) :
    cmd = f"{Command}"
    self.STM32.write(bytes(cmd,encoding='utf-8'))
  def waitResponse (self , Timeout) :
    timestamp = time.time()
    while (self.STM32.in_waiting == 0) :
      time.sleep(1)
      if ( timestamp - time.time() > Timeout ) :

        print ("No return from LowLevel")
        self.EndCommunication()
        return
    response = self.STM32.read_until('\n'.encode('utf-8'))
    response =response.decode('utf-8')
    observation = json.loads(response)
    return observation
  def EndCommunication (self) :
    self.STM32.close()


In [ ]:
class RobotOrientation (gym.Env) :
  def __init__(self , Port , Baudrate , WheelRadius , RobotSpacing) -> None:
    super().__init__()
    self.Port = Port
    self.Baudrate = Baudrate
    self.WheelRadius = WheelRadius
    self.RobotSpacing = RobotSpacing
    self.STM32 = LowLevelCommunication ( self.Port , self.Baudrate )

# Test for raw data communication :
# Robot = RobotOrientation("COM13" , 115200 , 10 , 15)
# ===> Working Fine with Local runtime connected.

    self.action_space = gym.spaces.Discrete( len(STP_ACTIONS_MAP) )
    # Observations are CurrentRobotAngle, CurrentRightWheelVelocity , CurrentLeftWheelvelocity
    self.observation_space = gym.spaces.Box(
            low=np.array([-180, -np.inf , -np.inf]),
            high=np.array([180,  np.inf ,  np.inf]),
            dtype=np.float32
        )

    self.RightWheelTicksPrev = 0
    self.LeftWheelTicksPrev = 0
    self.CurrentAngle = 0

  def __del__ (self) :
    self.STM32.EndCommunication()

  def step (self , action) :
    for counter in range (5) :
      self.STM32.sendCommand (5)
    obs = self.STM32.waitResponse(STM_ResponseTimeout)

    result = (obs['STM32TimeStamp'] , obs ['Terminated'] , obs ['CurrentRobotAngle'] , obs ['CurrentRightWheelVelocity'] , obs ['CurrentRightWheelVelocity'] )
    return (result)

# Testing The data exchange :
Robot = RobotOrientation("COM13" , 115200 , 10 , 15)
Robot.step(10)

Open


(255.0, 0.0, 92.300003, -233.300003, -233.300003)